In [1]:
%matplotlib inline

import os
import glob
import imageio

from math import atan2, degrees

import numpy as np
import pandas as pd

import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib.pyplot import cm
from matplotlib.colors import ListedColormap

from PIL import Image

from scipy.ndimage import gaussian_filter

# Plot hetmaps over selected frame without data from rejected sessions.

In [2]:
DISPLAY_SIZE = (320,200)
extent = [0, DISPLAY_SIZE[0], DISPLAY_SIZE[1], 0]

cmap = cm.jet
my_cmap = cmap(np.arange(cmap.N))
# Set alpha
my_cmap[:,-1] = np.linspace(0, 1, cmap.N)
# Create new colormap
cm_jet = ListedColormap(my_cmap)

def visual_span_from_distance(distance):
    h = 324 # Monitor height in cm
    d = distance # Distance between monitor and participant in cm
    r = 800 # Vertical resolution of the monitor
    size_in_deg = 5. # The stimulus size in pixels
    # Calculate the number of degrees that correspond to a single pixel. This will
    # generally be a very small value, something like 0.03.
    deg_per_px = degrees(atan2(.5*h, d)) / (.5*r)
    # Calculate the size of the stimulus in degrees
    size_in_px = size_in_deg / deg_per_px
    return size_in_px

def normalize(data):
    return ( data - data.min() ) / ( data.max() - data.min() )

def get_data(mediafile):
    media = imageio.get_reader("../Dataset/Mahnob/data/media_24/{}.avi".format(mediafile))
    
    valence_fixations_heatmaps = np.load("data/valence_fixation_heatmap_{}.npz".format(mediafile))
    valence_emotion_heatmaps = np.load("data/valence_emotion_heatmap_{}.npz".format(mediafile))
    
    arousual_fixations_heatmaps = np.load("data/arousal_fixation_heatmap_{}.npz".format(mediafile))
    arousal_emotion_heatmaps = np.load("data/arousal_emotion_heatmap_{}.npz".format(mediafile))
    
    return (media,
            valence_fixations_heatmaps, valence_emotion_heatmaps, 
            arousual_fixations_heatmaps, arousal_emotion_heatmaps)


##
## PLOT FUNCTIONS
##

def create_axes(sub=(1,1)):
    
    dpi = 100
    ncols=sub[0]
    nrows=sub[1]
    # determine the figure size in inches
    figsize = (1280*ncols/dpi, 800*nrows/dpi)
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, dpi=dpi)
    
    extent = [0, DISPLAY_SIZE[0], DISPLAY_SIZE[1], 0]

    if not isinstance(axes, type(np.array)): 
        axes = np.array(axes) 
    
    for ax in axes.flatten(): 
        ax.axis(extent)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    
    return axes.flatten()

def plot_images(axes, frames):
    images = get_media_frame(media, frames)
    for ax, im in zip(axes, images):
        ax.imshow(im, origin='lower')
    
def plot_coordinates(axes, data, frames, retax=False):
    frame_coordinates = get_frame_coordinates(data, frames)
    for ax, coord in zip(axes, frame_coordinates):
        for c in coord:
            x, y, _, _ = c
            ax.scatter(x, y, s=10, alpha=.8, c='cyan')

def plot_fixations(axes, data, frames):
    frame_fixations = get_frame_fixations(data, frames)
    for ax, fix in zip(axes, frame_fixations):
        for f in fix:
            x, y, d, distance, _ = f
            ax.scatter(x, y, s=2, alpha=.8, c='orange')
            r = visual_span_from_distance(distance) / 4
            circle = plt.Circle((x, y), 1e-2*d, alpha=.8, color='orange', fill=False)
            ax.add_artist(circle)

def get_heatmap_frames_over_threashold(heatmaps, tr):
    sampled_heatmaps = []
    for index, heatmap in enumerate(heatmaps):
        if heatmap.max() >= tr:
            sampled_heatmaps.append((index, heatmap))
    return sampled_heatmaps

In [3]:
def fuse_image(frame, heatmap):
    image = Image.fromarray(frame).convert("RGBA")
    hm =  cm_jet(heatmap, bytes=True)
    hm_image = Image.fromarray(hm)
    image.paste(hm_image, (0,0), hm_image)
    return np.array(image)

def export_to_video(type_of_data, media, sampled_heatmaps, tr):
    fps = media.get_meta_data()['fps']
    filename = "video/{}_{}_{}.avi".format(type_of_data, mediafile, tr)
    with imageio.get_writer(filename, fps=fps) as output_media:
        for frame_index, heatmap in sampled_heatmaps:
            try:
                media.set_image_index(frame_index)
                frame = media.get_next_data()
                image = fuse_image(frame, heatmap)
                output_media.append_data(image)
            except IndexError as e:
                print(e)
    
def export_to_gif(type_of_data, media, sampled_heatmaps, tr):
    filename = "gif/{}_{}_{}.gif".format(type_of_data, mediafile, tr)
    with imageio.get_writer(filename, mode='I', fps=10) as output_media:
        for frame_index, heatmap in sampled_heatmaps:
            try:
                media.set_image_index(frame_index)
                frame = media.get_next_data()
                image = fuse_image(frame, heatmap)
                output_media.append_data(image)
            except IndexError as e:
                print(e)

In [4]:
for mediafile in [30, 53, 69, 90, 111]:
    
    (media, 
     valence_fixations_heatmaps, valence_emotion_heatmaps, 
     arousual_fixations_heatmaps, arousal_emotion_heatmaps) = get_data(mediafile)
    
    for tr in [.2, .4, .6, .8]:
        
        heatmaps = valence_emotion_heatmaps['heatmaps']
        sampled_heatmaps = get_heatmap_frames_over_threashold(heatmaps, tr)

        if len(sampled_heatmaps) > 24 * 4:
            export_to_video('valence', media, sampled_heatmaps, tr)
        else:
            export_to_gif('valence', media, sampled_heatmaps, tr)
            
        heatmaps = arousal_emotion_heatmaps['heatmaps']
        sampled_heatmaps = get_heatmap_frames_over_threashold(heatmaps, tr)

        if len(sampled_heatmaps) > 24 * 4:
            export_to_video('arousal', media, sampled_heatmaps, tr)
        else:
            export_to_gif('arousal', media, sampled_heatmaps, tr)